In [1]:
# lets download the annotations from http://mscoco.org/dataset/#download
import os
import json
coco = json.load(open('../../Data/neural_talk_data/caption_datasets/dataset_coco.json', 'r'))

In [2]:
import h5py
import numpy as np

def create_dataset(h5py_file, string_list):
    data = np.array([['I', 'am', 'a', 'sentence'], ['another', 'sentence']], dtype=object)
    string_dt = h5py.special_dtype(vlen=str)
    h5py_file.create_dataset("sentences", data=data, dtype=string_dt)
    
h5File.close()
h5File=h5py.File('xxx.h5','w')

create_dataset(h5File)
h5File.close()


NameError: name 'h5File' is not defined

In [16]:
import numpy as np
aa =  np.eye(5)
print aa[np.array([1,3,1])]

[[ 0.  1.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.]]


# Generator annotation json (may be done already)

In [17]:
import sys
import json
import os
import shutil
import numpy as np
import h5py

sys.path.insert(0, '..')
from utils.local_utils import *

Data_root = os.path.join('..','..', 'Data', 'bladder')
save_root = os.path.join('..','..', 'Data', 'bladder_data')

annotation_folder = os.path.join(Data_root, 'Annotation') 
Img_folder = os.path.join(Data_root, 'Img') 

output_json = os.path.join(save_root,'total_anno.json') 

anno_filelist, anno_filenames = getfilelist(annotation_folder,['.json'] )

total_num = len(anno_filelist)
print total_num

total_json = []
test_num = total_num

# pool for one hot label
n_labels = 4
one_hot = np.eye(n_labels).astype(np.int)


hdf5_filepath = os.path.join(save_root, 'images_caption.h5')
hdf5_ref_filepath = os.path.join(save_root, 'images_caption_ref.json')

def get_cnn_img(thisimg, shape, norm=False):
    ''' 
    Parameters:
    -----------
    shape, (row, col, channel).
    norm: if you preder to normalize iter

    Returns:
    ----------
    img: Tensor of shape(channel, row, col)

    '''
    if isinstance(thisimg, np.ndarray):
        img = thisimg
    elif type(thisimg) is str:
        img = imread(thisimg)
    img  = pre_process_img(img, yuv = False, norm= norm)  #(img, yuv = False,norm= local_norm)

    img =  imresize_shape(img,shape)
    # transpose the img to order (channel, row, col)
    img = np.transpose(img, (2,0,1))
    return img

def parse_anno(anno_dict_list):

    str_list = []
    if type(anno_dict_list) is not list:
        anno_dict_list = [anno_dict_list]
    for anno_dict in anno_dict_list:         
        thisstr = ''
        for k, v in anno_dict.iteritems():
            thisstr = thisstr + ' '+ k.title() +': '
            thisstr  = thisstr + ' ' + v[1]
        str_list.append(thisstr)
    return str_list


with h5py.File(hdf5_filepath,'w') as ImgHdf5:
    #dset_img = ImgHdf5.create_dataset("images", (total_num, 3, 224, 224), dtype='uint8')
    dset_img = ImgHdf5.create_dataset("images", (test_num, 3, 224, 224), dtype='uint8')

    dset_conclusion = ImgHdf5.create_dataset("conclusion", (test_num, n_labels), dtype='int')

    string_dt = h5py.special_dtype(vlen=str)

    sentence_list  = []
    filename_list = []
    filename_dict = {}

    for idx, (fp, fn) in enumerate(zip(anno_filelist, anno_filenames )): 
        if idx == test_num:
            break

        img_name = fn + '.png'
        thisimg = imread(os.path.join(Img_folder, img_name))
        filename_dict[fn] = idx

        new_img  = get_cnn_img(thisimg, (224, 224,3), norm=False).astype(np.uint8)

        with open(fp) as data_file:    
            thisjson = json.load(data_file)
        sentence_list.append(parse_anno(thisjson)) 
        filename_list.append(fn)

        this_conclusion = thisjson[0]['conclusion'][0] #not beautiful
        dset_img[idx] = new_img
        dset_conclusion[idx] = one_hot[this_conclusion]


    sentence_data = np.array(sentence_list, dtype=object)
    ImgHdf5.create_dataset("sentences", data=sentence_data, dtype=string_dt)

    filename_data = np.array(filename_list, dtype= object)
    ImgHdf5.create_dataset("filenames", data=filename_data, dtype=string_dt)
    ImgHdf5.close()

    with open(hdf5_ref_filepath,'w') as f:
        json.dump(filename_dict, f)


50008


# Load and test the hdf5 file

In [2]:
import h5py

import sys
import json
import os
import shutil
import numpy as np
import h5py

sys.path.insert(0, '..')
from utils.local_utils import *

Data_root = os.path.join('..','..', 'Data', 'bladder')
annotation_folder = os.path.join(Data_root, 'Annotation') 
Img_folder = os.path.join(Data_root, 'Img') 

output_json = os.path.join(Data_root,'total_anno.json') 

anno_filelist, anno_filenames = getfilelist(annotation_folder,['.json'] )

total_num = len(anno_filelist)
print total_num

total_json = []
test_num = total_num

# pool for one hot label
n_labels = 4
one_hot = np.eye(n_labels).astype(np.int)


hdf5_filepath = os.path.join(Data_root, 'images_caption.h5')


with h5py.File(hdf5_filepath,'r') as testh5:
    imgs = testh5['images']
    sents = testh5['sentences']
    labs  = testh5['conclusion']
    fns   = testh5['filenames']

    print imgs.shape
    print sents.shape
    print sents.value[[1,3,1,4]]
    print labs.value[[1,3,1,4]]



50008
(50008, 3, 224, 224)
(50008, 5)
[[ ' Polarity:  Polarity is negligibly lost. Nucleoli:  The nucleoli of nuclei are inconspicuous. Nuclear_Feature:  The nuclei are moderately pleomorphic. Mitosis:  There are rarely mitotic figures throughout the tissue. Nuclear_Crowding:  Nuclei are moderately crowded together. Conclusion:  Lg / punlmp.'
  ' Polarity:  Basement membrane polarity is negligibly lost. Nucleoli:  The nucleoli of nuclei are inconspicuous. Nuclear_Feature:  Moderate pleomorphism of the nuclei can be seen. Mitosis:  Mitosis is rare. Nuclear_Crowding:  Moderate nuclear crowding is present. Conclusion:  Lg / punlmp.'
  ' Polarity:  Polarity of nuclei is negligibly lost. Nucleoli:  The nucleoli are mostly inconspicuous. Nuclear_Feature:  Moderate pleomorphism is present in the nuclei. Mitosis:  Mitotic figures are rare. Nuclear_Crowding:  The nuclei are moderately crowded. Conclusion:  Lg / punlmp.'
  ' Polarity:  Polarity is negligibly lost. Nucleoli:  The nuclei have inco

In [ ]:








print coco.keys()
print type(coco['images'])
print type(coco['dataset'])

In [11]:
print coco['images'][0]
print('---------------')
print coco['dataset']
print('---------------')
print(coco['images'][0].keys())
print('---------------')
print coco['images'][0]['sentences']
print('---------------')
print coco['images'][0]['split']

{u'sentids': [770337, 771687, 772707, 776154, 781998], u'filepath': u'val2014', u'filename': u'COCO_val2014_000000391895.jpg', u'imgid': 0, u'split': u'test', u'sentences': [{u'tokens': [u'a', u'man', u'with', u'a', u'red', u'helmet', u'on', u'a', u'small', u'moped', u'on', u'a', u'dirt', u'road'], u'raw': u'A man with a red helmet on a small moped on a dirt road. ', u'imgid': 0, u'sentid': 770337}, {u'tokens': [u'man', u'riding', u'a', u'motor', u'bike', u'on', u'a', u'dirt', u'road', u'on', u'the', u'countryside'], u'raw': u'Man riding a motor bike on a dirt road on the countryside.', u'imgid': 0, u'sentid': 771687}, {u'tokens': [u'a', u'man', u'riding', u'on', u'the', u'back', u'of', u'a', u'motorcycle'], u'raw': u'A man riding on the back of a motorcycle.', u'imgid': 0, u'sentid': 772707}, {u'tokens': [u'a', u'dirt', u'path', u'with', u'a', u'young', u'person', u'on', u'a', u'motor', u'bike', u'rests', u'to', u'the', u'foreground', u'of', u'a', u'verdant', u'area', u'with', u'a', u

In [ ]:
bladder = json.load(open('../../Data/bladder/caption_datasets/dataset_coco.json', 'r'))